# LoC thresholding notebook

This notebook loads a set of Z-tracks taken across an image volume from the lung-on-a-chip project and thresholds positive/negative expression of set markers based on a manual and unbiased assessment of the underlying images:

1. Load z-tracks, segmentation and images.
2. Initiate a Napari viewer with key bindings to annotate positive/negative expressing cells
3. Assess results
4. Quantify based on manually chosen thresholds

In [1]:
import btrack # import tracking software
import napari # image viewer
from skimage import io # image loader and viewer
import os # for filename manipulations
import glob # for more fn operations 
import pandas as pd # module for data org 
import matplotlib.pyplot as plt # module for plotting
import seaborn as sns # module for plotting
import numpy as np # for math operations
from tqdm.auto import tqdm # progress bar
sns.set(font='Nimbus Sans', style = 'white') # my favourite formatting :)
sns.set_palette('PiYG') # my favourite colours :D 

## 1. Load images and labels

In [2]:
input_file = "/home/dayn/data/homuncu_loc_temp/job_1a_files.txt"

with open(input_file, "r") as file:
    job1a_files = [line.strip() for line in file]
print('\n'.join(job1a_files))

DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295.tif
DAPI-SPC-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_246.tif
DAPI-SPC-PDPN-ZO1/Day7_breath/20x_21-12-029A_A3456_Multichannel Z-Stack_20220818_196.tif
DAPI-SPC-PDPN-ZO1/Day7_static/20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_121.tif
DAPI-NKX21-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_286.tif
DAPI-NKX21-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_235.tif
DAPI-NKX21-PDPN-ZO1/Day7_static/20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_113.tif


In [3]:
server_rootdir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_1/'
local_rootdir = '/home/dayn/data/homuncu_loc_temp'

In [4]:
for n, fn in tqdm(enumerate(job1a_files), total = len(job1a_files), 
                  desc = 'Progress through images in Job 1A'):
    if n == 0:# skip first instance
        continue
    # unique ID (from the code at the end of each fn, just used to store threshold info) 
    ID = fn.split('_')[-1].replace('.tif','')
    # set image fn on server
    image_fn = os.path.join(server_rootdir, fn)
    # load image
    print('Loading image, please be patient (usually takes couple mins)')
    %time image = io.imread(image_fn) # load image
    # set labels filename
    labels_fn = os.path.join(local_rootdir, fn.replace('.tif', '_z_tracks_masks.h5'))
    # load labels (tracks and segmentation)
    with btrack.io.HDF5FileHandler(labels_fn, 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
            tracks = reader.tracks
            segmentation = reader.segmentation
    # format tracks so napari understands
    napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim = 2)
    # set output directory as same as where labels are
    output_dir = os.path.dirname(labels_fn)
    # initiate napari with key bindings
    viewer = napari.Viewer(title = f'{fn}')
    viewer.add_image(image, channel_axis=-1)
    track_layer = viewer.add_tracks(napari_tracks, tail_length=0, visible = True)
    labels = viewer.add_labels(segmentation)
    labels.contour = 2 # this makes all the masks into outlines but might slow your computer down A LOT (comment out line if so) 
    
    @viewer.bind_key('q')
    def ch1_extract_positive_pixel_value(event=None):
        ### extract cell ID under cursor
        cell_ID = track_layer._get_value(position=viewer.cursor.position)
        ### get time frame
        time = viewer.cursor.position[0]
        try:
            positive_track = [track for track in tracks if track.ID == cell_ID][0]
        except:
            print('Looks like there was no identifiable track there')
        ### get index of cell instance of selected frame
        cell_index = positive_track.t.index(time)
        ### extract the pixel intensity of that segment
        px_int =  positive_track['mean_intensity'][:,1][cell_index]
        ### write to output file
        with open(os.path.join(output_dir, f"{ID}_ch1_positive_details.csv"),"a") as f:
            f.write(f"{cell_ID, time, px_int}\n")
    @viewer.bind_key('z')
    def ch1_extract_negative_pixel_value(event=None):
        ### extract cell ID under cursor
        cell_ID = track_layer._get_value(position=viewer.cursor.position)
        ### get time frame
        time = viewer.cursor.position[0]
        try:
            positive_track = [track for track in tracks if track.ID == cell_ID][0]
        except:
            print('Looks like there was no identifiable track there')
        ### get index of cell instance of selected frame
        cell_index = positive_track.t.index(time)
        ### extract the pixel intensity of that segment
        px_int =  positive_track['mean_intensity'][:,1][cell_index]
        ### write to output file
        with open(os.path.join(output_dir, f"{ID}_ch1_negative_details.csv"),"a") as f:
            f.write(f"{cell_ID, time, px_int}\n")

    @viewer.bind_key('t')
    def ch2_extract_positive_pixel_value(event=None):
        ### extract cell ID under cursor
        cell_ID = track_layer._get_value(position=viewer.cursor.position)
        ### get time frame
        time = viewer.cursor.position[0]
        try:
            positive_track = [track for track in tracks if track.ID == cell_ID][0]
        except:
            print('Looks like there was no identifiable track there')
        ### get index of cell instance of selected frame
        cell_index = positive_track.t.index(time)
        ### extract the pixel intensity of that segment
        px_int =  positive_track['mean_intensity'][:,2][cell_index]
        ### write to output file
        with open(os.path.join(output_dir, f"{ID}_ch2_positive_details.csv"),"a") as f:
            f.write(f"{cell_ID, time, px_int}\n")
    @viewer.bind_key('b')
    def ch2_extract_negative_pixel_value(event=None):
        ### extract cell ID under cursor
        cell_ID = track_layer._get_value(position=viewer.cursor.position)
        ### get time frame
        time = viewer.cursor.position[0]
        try:
            positive_track = [track for track in tracks if track.ID == cell_ID][0]
        except:
            print('Looks like there was no identifiable track there')
        ### get index of cell instance of selected frame
        cell_index = positive_track.t.index(time)
        ### extract the pixel intensity of that segment
        px_int =  positive_track['mean_intensity'][:,2][cell_index]
        ### write to output file
        with open(os.path.join(output_dir, f"{ID}_ch2_negative_details.csv"),"a") as f:
            f.write(f"{cell_ID, time, px_int}\n")
            
    @viewer.bind_key('c')
    def crap_segmentation(event=None):
        with open(os.path.join(output_dir, f"{ID}_segmentation_needs_revisiting.csv"),"a") as f:
            f.write(f"segmentation? more like CRAPmentation")
    # Wait for the Napari viewer to be closed before continuing to the next iteration
    viewer.show(block = True)

Progress through images in Job 1A: 0it [00:00, ?it/s]

Loading image, please be patient (usually takes couple mins)


[INFO][2023/07/26 03:53:39 PM] Opening HDF file: /home/dayn/data/homuncu_loc_temp/DAPI-SPC-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_246_z_tracks_masks.h5...
[INFO][2023/07/26 03:53:39 PM] Loading tracks/obj_type_1
[INFO][2023/07/26 03:53:39 PM] Loading LBEP/obj_type_1
[INFO][2023/07/26 03:53:39 PM] Loading objects/obj_type_1 (52314, 5) (52314 filtered: None)


CPU times: user 1.28 s, sys: 3.47 s, total: 4.75 s
Wall time: 1min


[INFO][2023/07/26 03:53:42 PM] Loading segmentation (81, 2304, 2304)
[INFO][2023/07/26 03:53:42 PM] Closing HDF file: /home/dayn/data/homuncu_loc_temp/DAPI-SPC-PDPN-ZO1/Day14_static/20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_246_z_tracks_masks.h5
/home/dayn/analysis/btrack/btrack/dataio.py:3: UserWarning: `btrack.dataio` has been deprecated. Please use `btrack.io` subpackage instead.
  warnings.warn(  # noqa: B028


...

UnboundLocalError: local variable 'positive_track' referenced before assignment

Looks like there was no identifiable track there


UnboundLocalError: local variable 'positive_track' referenced before assignment

Looks like there was no identifiable track there


UnboundLocalError: local variable 'positive_track' referenced before assignment